In [5]:
rdd = sc.textFile("/air_transits.csv")

In [3]:
from operator import add

In [6]:
#Q1 Compute the Total number of records
rdd.count()

123534969

In [7]:
#Q2 Find total number of flights per year
data = rdd.map(lambda line: line.split(","))
data.map(lambda x:(x[0],1)).reduceByKey(add).sortByKey().collect()

[(u'1987', 1311826), (u'1988', 5202096), (u'1989', 5041200), (u'1990', 5270893), (u'1991', 5076925), (u'1992', 5092157), (u'1993', 5070501), (u'1994', 5180048), (u'1995', 5327435), (u'1996', 5351983), (u'1997', 5411843), (u'1998', 5384721), (u'1999', 5527884), (u'2000', 5683047), (u'2001', 5967780), (u'2002', 5271359), (u'2003', 6488540), (u'2004', 7129270), (u'2005', 7140596), (u'2006', 7141922), (u'2007', 7453215), (u'2008', 7009728)]

In [9]:
#Q3 Find the plane with the most number of flights. Each plane has a unique TailNum.
import re
data.filter(lambda x:re.match(r'N\d*', x[10])). \
    filter(lambda x:x[10] != 'NA'). \
    map(lambda x: (x[10],1)).reduceByKey(add). \
    map(lambda x:(x[1],x[0])).sortByKey(False). \
    take(1)

[(34526, u'N528')]

### Assume standard tail number is 'Nxxx' 

In [10]:
#Q4 Compute the total flight time of each plane.
data.filter(lambda x:x[10]!='NA'). \
    filter(lambda x:x[13]!='NA'). \
    map(lambda x:(x[10],int(x[13]))). \
    reduceByKey(add). \
    collect()

[(u'', 2901), (u'N4TXAA', 310980), (u'N410FJ', 217746), (u'N976UA', 816797), (u'N304SW', 836651), (u'N551AA', 2119003), (u'N255AY', 48549), (u'N829US', 958242), (u'N286AS', 49822), (u'-N906M', 106592), (u'N504AU', 1847877), (u'N338MQ', 213959), (u'N908H', 963279), (u'N937MC', 1035812), (u'N475A1', 178002), (u'N606\ufffd1', 230163), (u'N115UW', 717036), (u'N789', 1121144), (u'N315@@', 224601), (u'N480A1', 192199), (u'N613\ufffd\ufffd', 18452), (u'N7AUAA', 48059), (u'N965TW', 272384), (u'N26208', 1306606), (u'-N323M', 114868), (u'N7BEAA', 69095), (u'N281US', 140874), (u'N37422', 89010), (u'N137AA', 1185957), (u'N580A1', 197368), (u'N826AE', 612060), (u'N524\ufffd\ufffd', 239494), (u'N2BRA1', 147394), (u'N601', 1585987), (u'N627US', 23991), (u'N911TW', 1141828), (u'N631AE', 570885), (u'N2DJAA', 937171), (u'N378AA', 839387), (u'N563A1', 188656), (u'N5BRA1', 139689), (u'N6701', 1640332), (u'N868AA', 574706), (u'N512AE', 623307), (u'N987Z', 795378), (u'N2AFAA', 1093587), (u'N459UW', 380715),

In [15]:
#Q5 Find the busiest airport (in terms of number of departures) for each year.
busyairport = data\
        .map(lambda x: (x[16] +','+ x[0], 1)) \
        .reduceByKey(add)
years = [str(i) for i in range(1987,2009)]

for year_ in years:
    result = busyairport.filter(lambda x: x[0].split(',')[1]==year_).max(lambda x: x[1])
    print result

(u'ORD,1987', 67216)
(u'ORD,1988', 271494)
(u'ORD,1989', 261259)
(u'ATL,1990', 269397)
(u'ORD,1991', 263145)
(u'ORD,1992', 275185)
(u'ORD,1993', 286055)
(u'ORD,1994', 277741)
(u'ORD,1995', 293422)
(u'ORD,1996', 296234)
(u'ORD,1997', 297490)
(u'ORD,1998', 294896)
(u'ORD,1999', 298251)
(u'ORD,2000', 296771)
(u'ORD,2001', 341284)
(u'ORD,2002', 327388)
(u'ATL,2003', 368743)
(u'ATL,2004', 418072)
(u'ATL,2005', 433678)
(u'ATL,2006', 407971)
(u'ATL,2007', 413851)
(u'ATL,2008', 414513)

In [12]:
#Q6 Find the airline with the highest average delay in 2006.           
filterdata = data\
        .filter(lambda x: x[0] == '2006' and x[15] != 'NA')\
        .map(lambda x: (x[8],int(x[15])))\
        .aggregateByKey((0,0), lambda a,b: (a[0] + b,a[1] + 1),lambda a,b: (a[0] + b[0], a[1] + b[1]))

In [13]:
filterdata.mapValues(lambda v: v[0]/v[1]).sortByKey(False).take(1)

[(u'YV', 14)]

### Assume delay refers to only departure delay